In [1]:
import torch
import os
import shutil
from PIL import Image, ImageOps
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:13<00:00, 3.73MB/s]


In [3]:
#!yolo predict model=yolov8n.pt source='Images/'

Ultralytics YOLOv8.0.182  Python-3.10.10 torch-2.0.1+cpu CPU (AMD Ryzen 5 5600X 6-Core Processor)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

Traceback (most recent call last):
  File "C:\Users\darki\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\darki\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\Scripts\yolo.exe\__main__.py", line 7, in <module>
    sys.exit(entrypoint())
  File "C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\lib\site-packages\ultralytics\cfg\__init__.py", line 445, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\lib\site-packages\ultralytics\engine\model.py", line 236, in predict
    return self.predictor.predict_cli(sou

In [8]:
def loadImagesPaths(path):
    image_paths = []
    # Get a list of image file names in the directory
    image_files = [f for f in os.listdir(path) if f.endswith((".jpg", ".png", ".jpeg"))]
    for image_file in image_files:
        image_paths.append(os.path.join(path, image_file))
    return image_paths

def predictImage(image_path):
    allBirdData = []
    results = model.predict(source=image_path, save_crop=True, project="boxes", name="prediction")
    for index in range(0, len(results[0].boxes.data)):
        # IS IT A BIRD?
        if (results[0].boxes.data[index][5] == 14):
            # Extract and parse data
            tx = torch.ceil(results[0].boxes.data[index][0]).item()
            ty = torch.ceil(results[0].boxes.data[index][1]).item()
            tw = torch.ceil(results[0].boxes.data[index][2]).item()
            th = torch.ceil(results[0].boxes.data[index][3]).item()
            score = results[0].boxes.data[index][4].item()
            birdData = [tx, ty, tw, th, score]

            allBirdData.append(birdData)
    return allBirdData

In [87]:
#target_size 224/224
def cropImages(path, target_size):
    prediction_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    imageNumber = 0
    for prediction_dir in prediction_dirs:
        # Construct the path to the "crop" folder within the prediction directory
        crop_folder = os.path.join(path, prediction_dir, "crops/bird/")

        # Ensure the "crop" folder exists
        if not os.path.exists(crop_folder):
            continue  # Skip if "crop" folder is not found

        # Get a list of image files in the "crop" folder
        image_files = [f for f in os.listdir(crop_folder) if f.endswith((".jpg", ".png", ".jpeg"))]

        for image_file in image_files:
            imageNumber += 1
            # Construct the full path to the image file
            image_path = os.path.join(crop_folder, image_file)

            image = Image.open(image_path)
            paddedImage = padImage(image, target_size)
            res_image = paddedImage.resize(target_size, Image.LANCZOS)
            # Save the image
            res_image.save(f"cropped_images/bird{imageNumber}.png")

def padImage(image, targetSize):
        imageWidth, imageHeight = image.size
        #AR = aspect ratio
        currentAspectRatio = imageWidth/imageHeight
        targetAspectRatio = targetSize[0]/targetSize[1]
        if currentAspectRatio > targetAspectRatio:  # Vertical padding
            verticalPadding = int(((imageWidth/targetAspectRatio)-imageHeight)/2)
            #print("vertical padding: ", verticalPadding)
            padding = (0, verticalPadding, 0, verticalPadding)
            paddedImage = ImageOps.expand(image, padding, fill="white")

        elif currentAspectRatio < targetAspectRatio:    # Horizontal padding
            horizontalPadding = int(((targetAspectRatio*imageHeight)-imageWidth)/2)
            #print("horizontal padding: ", horizontalPadding)
            padding = (horizontalPadding, 0, horizontalPadding, 0)
            paddedImage = ImageOps.expand(image, padding, fill="white")

        else:
            paddedImage = image

        #paddedImage.show()
        #print("Og size: " , image.size, "AR: ", currentAspectRatio)
        #print("Pad size: " , paddedImage.size, "AR: ", paddedImage.size[0]/paddedImage.size[1])
        #print("Tar size: " , targetSize, "AR: ", targetAspectRatio, "\n")
        return paddedImage

In [58]:
def deleteimages(path):
    # Get a list of all files and subdirectories in the directory
    directory_contents = os.listdir(path)

    # Iterate through the contents and remove them
    #print(directory_contents)
    for item in directory_contents:
        item_path = os.path.join(path, item)
        if os.path.isfile(item_path):
            # Remove files
            os.remove(item_path)
        elif os.path.isdir(item_path):
            # Remove directories and their contents (recursively)
            shutil.rmtree(item_path)

In [88]:
def runClassifcation():
    deleteimages("cropped_images/")
    image_paths = loadImagesPaths(path="images/")
    predictions = []
    for image in image_paths:
        predictions.append(predictImage(image))
    cropImages(path="boxes/", target_size=(284, 224))
    deleteimages("boxes/")

runClassifcation()

['bird1.png', 'bird2.png', 'bird3.png', 'bird4.png']


image 1/1 C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\images\fagel.jpg: 448x640 1 bird, 309.0ms
Speed: 1.0ms preprocess, 309.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to boxes\prediction

image 1/1 C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\images\Porg.jpg: 640x384 1 bird, 267.0ms
Speed: 2.0ms preprocess, 267.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to boxes\prediction2

image 1/1 C:\Users\darki\Documents\Skol dokument\D7057E\yolo_v8\images\two_birds.jpg: 352x640 2 birds, 235.0ms
Speed: 1.0ms preprocess, 235.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to boxes\prediction3


['prediction', 'prediction2', 'prediction3']
